In [2]:
import os
import numpy as np
from bct import modularity

from signed_modularity import modularity_louvain_und_sign
from itertools import combinations

In [3]:
def load_group_files(base_url, group_path):
    path = '{}/{}/'.format(base_url, group_path)
    file_names = sorted(os.listdir(path))
    files = ['{}{}'.format(path, elem) for elem in file_names]
    return files

def load_matrix_subjects(base_url, group_path):
    group_files = load_group_files(base_url, group_path)
    return _load_matrix_subject_with_files(group_files)

def _load_matrix_subject_with_files(files):
    subjects = []
    for file in files:
        mat = np.loadtxt(file, delimiter=' ')
        np.fill_diagonal(mat, 0)
        subjects.append(mat)
    return np.array(subjects)

def nearest_neighbour_edge_centrality(edge, matrix):
    degree_a = np.sum(matrix[edge[0],])
    degree_b = np.sum(matrix[edge[1],])
    w_ab = matrix[edge[0], edge[1]]

    return (degree_a + degree_b - 2 * w_ab) / (np.abs(degree_a - degree_b) + 1) * w_ab


In [4]:
def measure_edge_weight(aff, mat):
    inter_weight = 0
    intra_weight = 0

    for e in edge_combo:
        if aff[e[0]] == aff[e[1]]:
            intra_weight += mat[e]
        else:
            inter_weight += mat[e]

    inter_ratio = inter_weight / np.sum(mat) * 2
    intra_ratio = intra_weight / np.sum(mat) * 2

    print(f'inter-edge ratio: {inter_ratio:.4f}, intra-edge ratio: {intra_ratio:.4f}')
    # return inter_ratio, intra_ratio

In [5]:
data_type = 'correlation'
datasets = ['adolescents', 'children', 'eyesclosed', 'male', 'female', 'complete_abide']

n_nodes = 116
edge_combo = list(combinations(range(n_nodes), 2))

In [6]:
idx = 3

In [7]:
dataset = datasets[idx]

diff_mat = np.loadtxt(f"../abide_datasets/{data_type}_diff_mat/{dataset}/td_asd_diff_mat_whole_dataset.txt")

td_subjects = load_matrix_subjects(f'../abide_datasets/{data_type}/{dataset}', 'td')
asd_subjects = load_matrix_subjects(f'../abide_datasets/{data_type}/{dataset}', 'asd')

td_mean = np.mean(td_subjects, axis=0)
asd_mean = np.mean(asd_subjects, axis=0)

In [8]:
seed = 5

In [9]:
ci, q = modularity_louvain_und_sign(td_mean, qtype='sta', seed=seed)
smp_ci, smp_q = modularity_louvain_und_sign(td_mean, qtype='smp', seed=seed)
orig_ci, orig_q = modularity.modularity_louvain_und_sign(td_mean, qtype='sta', seed=seed)
orig_smp_ci, orig_smp_q = modularity.modularity_louvain_und_sign(td_mean, qtype='smp', seed=seed)

In [10]:
ci

array([4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 2, 2, 4, 4, 4, 4, 2, 2,
       2, 2, 2, 2, 2, 2, 4, 4, 2, 2, 4, 4, 2, 2, 4, 4, 5, 5, 4, 4, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 1, 1, 1, 1, 4, 4, 2, 2,
       1, 2, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 2, 2, 2, 2,
       2, 2, 5, 5, 5, 5, 5, 5, 3, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5])

In [11]:
print('sta')
print('ours:')
measure_edge_weight(ci, td_mean * (td_mean < 0))
measure_edge_weight(ci, td_mean * (td_mean > 0))
print('olaf')
measure_edge_weight(orig_ci, td_mean * (td_mean < 0))
measure_edge_weight(orig_ci, td_mean * (td_mean > 0))
print('\nsmp')
print('ours:')
measure_edge_weight(smp_ci, td_mean * (td_mean < 0))
measure_edge_weight(smp_ci, td_mean * (td_mean > 0))
print('olaf')
measure_edge_weight(orig_smp_ci, td_mean * (td_mean < 0))
measure_edge_weight(orig_smp_ci, td_mean * (td_mean > 0))

sta
ours:
inter-edge ratio: 0.9630, intra-edge ratio: 0.0370
inter-edge ratio: 0.2845, intra-edge ratio: 0.7155
olaf
inter-edge ratio: 0.9630, intra-edge ratio: 0.0370
inter-edge ratio: 0.2845, intra-edge ratio: 0.7155

smp
ours:
inter-edge ratio: 0.9066, intra-edge ratio: 0.0934
inter-edge ratio: 0.2155, intra-edge ratio: 0.7845
olaf
inter-edge ratio: 0.9066, intra-edge ratio: 0.0934
inter-edge ratio: 0.2155, intra-edge ratio: 0.7845


In [12]:
ci, q = modularity_louvain_und_sign(asd_mean, qtype='sta', seed=seed)
smp_ci, smp_q = modularity_louvain_und_sign(asd_mean, qtype='smp', seed=seed)
orig_ci, orig_q = modularity.modularity_louvain_und_sign(asd_mean, qtype='sta', seed=seed)
orig_smp_ci, orig_smp_q = modularity.modularity_louvain_und_sign(asd_mean, qtype='smp', seed=seed)

In [13]:
print('sta')
print('ours:')
measure_edge_weight(ci, td_mean * (td_mean < 0))
measure_edge_weight(ci, td_mean * (td_mean > 0))
print('olaf')
measure_edge_weight(orig_ci, td_mean * (td_mean < 0))
measure_edge_weight(orig_ci, td_mean * (td_mean > 0))
print('\nsmp')
print('ours:')
measure_edge_weight(smp_ci, td_mean * (td_mean < 0))
measure_edge_weight(smp_ci, td_mean * (td_mean > 0))
print('olaf')
measure_edge_weight(orig_smp_ci, td_mean * (td_mean < 0))
measure_edge_weight(orig_smp_ci, td_mean * (td_mean > 0))

sta
ours:
inter-edge ratio: 0.9099, intra-edge ratio: 0.0901
inter-edge ratio: 0.2382, intra-edge ratio: 0.7618
olaf
inter-edge ratio: 0.9099, intra-edge ratio: 0.0901
inter-edge ratio: 0.2382, intra-edge ratio: 0.7618

smp
ours:
inter-edge ratio: 0.8945, intra-edge ratio: 0.1055
inter-edge ratio: 0.2189, intra-edge ratio: 0.7811
olaf
inter-edge ratio: 0.8945, intra-edge ratio: 0.1055
inter-edge ratio: 0.2189, intra-edge ratio: 0.7811
